In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
data_path = '/content/drive/MyDrive/IPD/code/image/hate_symbols/data'
train_path = '/content/drive/MyDrive/IPD/code/image/hate_symbols/train'
valid_path = '/content/drive/MyDrive/IPD/code/image/hate_symbols/valid'

In [ ]:
# for dir in os.listdir(data_path):
#   new_folder_path = os.path.join(train_path, dir)
#   new_folder_path = os.path.join(valid_path, dir)
#   os.makedirs(new_folder_path, exist_ok=True)

In [ ]:
# from PIL import Image
# import os

# for dir in os.listdir(data_path):
#   new_folder_path = os.path.join(train_path, dir)
#   for file in os.listdir(os.path.join(data_path,dir)):
#     try:
#       img = Image.open(os.path.join(data_path,dir,file))
#       img = img.resize((224,224))
#       rgb_img = img.convert('RGB')
#       rgb_img.save(os.path.join(new_folder_path,file.split('.')[0]+'.jpg'))
#     except:
#       pass

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np

num_classes = 5
image_size = (224, 224)
batch_size = 8
epochs = 10

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=image_size + (3,))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

steps_per_epoch = len(train_generator)//batch_size
validation_steps = len(val_generator)//batch_size

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=validation_steps
)

Found 1043 images belonging to 5 classes.
Found 258 images belonging to 5 classes.
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 158s 7s/step - accuracy: 0.4180 - loss: 3.5732 - val_accuracy: 0.5938 - val_loss: 3.7232
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 103s 6s/step - accuracy: 0.5497 - loss: 2.1819 - val_accuracy: 0.0625 - val_loss: 828.8075
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 96s 6s/step - accuracy: 0.5457 - loss: 1.2973 - val_accuracy: 0.4062 - val_loss: 1707.5105
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 101s 6s/step - accuracy: 0.6573 - loss: 1.2152 - val_accuracy: 0.6562 - val_loss: 2.1774
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 99s 6s/step - accuracy: 0.6805 - loss: 1.1069 - val_accuracy: 0.6250 - val_loss: 1.7910
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 101s 6s/step - accuracy: 0.7083 - loss: 1.0729 - val_accuracy: 0.6250 - val_loss: 1.5045
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 101s 6s/step - accuracy: 0.7586 - loss: 0.9042 - val_accuracy: 0.5312 - val_loss: 1.7347
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 101s 6s/step - accuracy: 0.6903 - loss: 0.9976 - val_accuracy: 0.6562 - val_loss:

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


16/16 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.5768 - loss: 1.1729 - val_accuracy: 0.5000 - val_loss: 1.6920
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 113s 6s/step - accuracy: 0.5390 - loss: 1.1935 - val_accuracy: 0.6250 - val_loss: 3.7649


In [ ]:
model.save('symbol_detector_resnet.keras')

In [ ]:
class_indices = train_generator.class_indices
index_to_class = {v: k for k, v in class_indices.items()}
print(index_to_class)

{0: 'cut_throat_gesture', 1: 'finger_gun_to_the_head', 2: 'middle_finger', 3: 'slanted_eyes_gesture', 4: 'swastika'}


In [ ]:
def predict_single_image(image_path, model, image_size):
    img = load_img(image_path, target_size=image_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    print(predictions)
    predicted_class = np.argmax(predictions, axis=1)

    return predicted_class, predictions

image_path = 'hate_symbol.jpeg'
predicted_class, predictions = predict_single_image(image_path, model, image_size)
print(f"Predicted Class: {predicted_class}")
print(f"Predictions: {predictions}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
[[4.4611324e-02 9.5517528e-01 2.0699757e-04 5.6824829e-06 7.3142149e-07]]
Predicted Class: [1]
Predictions: [[4.4611324e-02 9.5517528e-01 2.0699757e-04 5.6824829e-06 7.3142149e-07]]
